In [ ]:
import yfinance as yf
import pandas as pd
import time
import findspark
import os

In [ ]:
findspark.init()

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder \
    .appName("StockData") \
    .getOrCreate()

In [ ]:
stock_symbol = 'NVDA'
stock = yf.Ticker(stock_symbol)
data = stock.history(period='1d', interval='1m').iloc[-2]
columns = ["Metric", "Value"]
df = spark.createDataFrame(data.reset_index(),columns)
df.show()

In [ ]:
stock_symbol = 'NVDA'  # Change to your desired stock symbol
root_path = os.getcwd()
hdfs_path = "hdfs://localhost:9000/project/stream/"
    
for i in range(2):  
    
    stock = yf.Ticker(stock_symbol)
    data = stock.history(period='1d', interval='1m').iloc[-2]
    columns = ["Metric", "Value"]
    df = spark.createDataFrame(data.reset_index(),columns)
    df.show()
    
    current_time = pd.Timestamp.now()
    timestamp_str = current_time.strftime('%Y%m%d%H%M%S')
    print(timestamp_str)
    
    if df.count() > 0:
        df.write.mode("overwrite").parquet(os.path.join(hdfs_path, f"{timestamp_str}.parquet"))
    else:
        print("DataFrame is empty, skipping write.")
    
    time.sleep(60)

spark.stop()

In [ ]:
parquet_file_path = os.path.join(output_folder,'20250527231100.parquet')  # Adjust the path as needed

# Read the Parquet file into a DataFrame
df = spark.read.parquet(parquet_file_path)

# Show the DataFrame
df.show()

# Stop the Spark session
spark.stop()